Obtenha uma tabela que contenha o id do pedido e o valor total do mesmo

In [2]:
SELECT * FROM orders LIMIT 1;
SELECT * FROM order_details LIMIT 1;
SELECT * FROM products LIMIT 1;
SELECT * FROM customers LIMIT 1;

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:03.412

order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country
10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,NULL,51100,France


order_id,product_id,unit_price,quantity,discount
10248,11,14.0,12,0.0


product_id,product_name,supplier_id,category_id,quantity_per_unit,unit_price,units_in_stock,units_on_order,reorder_level,discontinued
1,Chai,8,1,10 boxes x 30 bags,18.0,39,0,10,1


customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NULL,12209,Germany,030-0074321,030-0076545


# Praticando INNER JOIN

## Informações:

```
SELECT * FROM orders;
SELECT * FROM order_details;
SELECT * FROM products;
SELECT * FROM customers;
```

- Em algum momento vocês vão usar:
    
    ```
    SUM((products.unit_price * order_details.quantity * order_details.discount)) + AVG(orders.freight) AS valor_total
    ```
    
- Em alguns momentos vocês vão usar:
    
    ```
    SUM((products.unit_price * order_details.quantity * (1 - order_details.discount))) + AVG(orders.freight) as total_order_price
    ```
    
- Em algum momento vocês vão usar:
    
    ```
      ROUND(
          SUM((products.unit_price * order_details.quantity * (1 - order_details.discount))) + AVG(orders.freight)
      ) as total_order_price
    ```
    
- Em algum momento vocês vão usar:
    
    ```
      (
          SUM((products.unit_price * order_details.quantity * (1 - order_details.discount))) + AVG(orders.freight)
      ) / COUNT(orders.order_id) as mean_order_price
    ```
    

1. .

1\. Obtenha uma tabela que contenha o id do pedido e o valor total do mesmo

In [20]:
SELECT order_details.order_id, 
SUM((order_details.quantity * order_details.discount)) + AVG(orders.freight) AS valor_total
FROM order_details 
INNER JOIN orders ON orders.order_id = order_details.order_id
GROUP BY order_details.order_id
LIMIT 1

(1 row(s) affected)

Total execution time: 00:00:00.866

order_id,valor_total
10248,32.380001068115234


<span style="color: rgb(51, 51, 51); font-family: Muli, &quot;Helvetica Neue&quot;, Arial, sans-serif; letter-spacing: -0.1px;">2. Obtenha uma lista dos 10 clientes que realizaram o maior número de pedidos, bem como o número de pedidos de cada, ordenados em ordem decrescente de nº de pedidos.</span>

In [17]:
SELECT customers.customer_id, customers.company_name
,COUNT(orders.order_id) AS num_pedidos
FROM orders 
INNER JOIN customers ON customers.customer_id = orders.customer_id
GROUP BY customers.customer_id
ORDER BY 3 DESC
LIMIT 10

(10 row(s) affected)

Total execution time: 00:00:00.844

customer_id,company_name,num_pedidos
SAVEA,Save-a-lot Markets,31
ERNSH,Ernst Handel,30
QUICK,QUICK-Stop,28
FOLKO,Folk och fä HB,19
HUNGO,Hungry Owl All-Night Grocers,19
RATTC,Rattlesnake Canyon Grocery,18
BERGS,Berglunds snabbköp,18
HILAA,HILARION-Abastos,18
BONAP,Bon app',17
FRANK,Frankenversand,15


3\. Obtenha uma tabela que contenha o id e o valor total do pedido e o nome do cliente que o realizou.

In [15]:
SELECT order_details.order_id, 
SUM((products.unit_price * order_details.quantity * (1 - order_details.discount))) + AVG(orders.freight) as total_order_price
, customers.company_name
, customers.contact_name
FROM order_details 
INNER JOIN products ON order_details.product_id = products.product_id
INNER JOIN orders ON orders.order_id = order_details.order_id
INNER JOIN customers ON customers.customer_id = orders.customer_id
GROUP BY order_details.order_id, customers.company_name, customers.contact_name
LIMIT 1


(1 row(s) affected)

Total execution time: 00:00:00.830

order_id,total_order_price,company_name,contact_name
10254,718.7549961358309,Chop-suey Chinese,Yang Wang


4\. Obtenha uma tabela que contenha o país do cliente e o valor da compra que ele realizou.

In [14]:
SELECT order_details.order_id, 
ROUND(
      SUM((products.unit_price * order_details.quantity * (1 - order_details.discount))) + AVG(orders.freight)
  ) as total_order_price
, customers.company_name
, customers.country
FROM order_details 
INNER JOIN products ON order_details.product_id = products.product_id
INNER JOIN orders ON orders.order_id = order_details.order_id
INNER JOIN customers ON customers.customer_id = orders.customer_id
GROUP BY order_details.order_id, customers.company_name, customers.country
LIMIT 4

(4 row(s) affected)

Total execution time: 00:00:00.866

order_id,total_order_price,company_name,country
10897,11439.0,Hungry Owl All-Night Grocers,Ireland
10637,2963.0,Queen Cozinha,Brazil
10383,1158.0,Around the Horn,UK
11074,251.0,Simons bistro,Denmark


5\. Obtenha uma tabela que contenha uma lista dos países dos clientes e o valor total de compras realizadas em cada um dos países. Ordene a tabela, na order descrescente, considerando o valor total de compras realizadas por país.

In [19]:
SELECT  customers.country
,ROUND(SUM((order_details.quantity * order_details.discount)) + AVG(orders.freight)) AS valor_total
FROM customers 
INNER JOIN orders ON customers.customer_id = orders.customer_id
INNER JOIN order_details ON order_details.order_id = orders.order_id
GROUP BY customers.country
ORDER BY 2 DESC
LIMIT 5

(5 row(s) affected)

Total execution time: 00:00:00.828

country,valor_total
USA,813.0
Germany,721.0
Austria,598.0
Brazil,348.0
Ireland,339.0


6\. Obtenha uma tabela com o valor médio das vendas em cada mês (ordenados do mês com mais vendas para o mês com menos vendas).

In [20]:
WITH base as (
SELECT  extract(MONTH FROM orders.order_date)::INTEGER AS "Mes"
,extract(YEAR FROM orders.order_date)::INTEGER AS "Ano"
,order_details.order_id
,(
      SUM((products.unit_price * order_details.quantity * (1 - order_details.discount))) + AVG(orders.freight)
  ) / COUNT(orders.order_id) as mean_order_price
FROM orders 
INNER JOIN order_details ON order_details.order_id = orders.order_id
INNER JOIN products ON order_details.product_id = products.product_id

GROUP BY order_details.order_id,
extract(MONTH FROM orders.order_date)::INTEGER 
,extract(YEAR FROM orders.order_date)::INTEGER )

SELECT "Mes","Ano", AVG(mean_order_price) AS "ValorMedia" FROM base
GROUP BY "Ano", "Mes"
ORDER BY 3 DESC
--LIMIT 5

(23 row(s) affected)

Total execution time: 00:00:00.841

Mes,Ano,ValorMedia
11,1996,875.7396903496298
2,1998,864.8733413790296
1,1997,820.0519468368522
3,1998,725.2787108734099
12,1996,690.779970704203
2,1997,663.9334034577668
4,1998,655.4541779294001
1,1998,623.950803782281
4,1997,623.5036895240738
12,1997,617.0268246040323
